<a href="https://colab.research.google.com/github/wajnryt/JerusMLDeepLearning2019/blob/master/ClassClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount data
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# DataSet object
import pandas as pd
import os
from torch.utils.data.dataset import Dataset
import imageio
import numpy as np

class InstanceDataset(Dataset):
    def __init__(self, basedir, transform=None):
        super().__init__()
        files = glob.glob(os.path.join(basedir ,'*','*','*.jpg'))
        self.data = pd.DataFrame([self._split_file(f) for f in files], 
                            columns=['class_id', 'file_path'])
        
    def _split_file(self, f):
        parts = f.split(os.sep)[-3:-1]
        return int(parts[0]), f   #label is originaly a str

    def __getitem__(self, index):
      dat = self.data.iloc[index]
      img = imageio.imread(dat['file_path'])
      img = np.resize(img,(3,128,128))
      return (torch.from_numpy(img.astype(np.float32)), dat['class_id'])
              
    def __len__(self):
        return len(self.data)

In [3]:
import glob
import torchvision.transforms as transforms

basedir = '/content/gdrive/My Drive/videos_2/yt_bb_detection_train'

# data_transforms = transforms.Compose([transforms.Resize(128,128), 
#                                       transforms.ToTensor(),
#                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

#Create DS
dataset = InstanceDataset(basedir)

len(np.unique(dataset.data['class_id']))  #how many classes

23

In [5]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

#Divide data to train/test
dataset = InstanceDataset(basedir)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])


batch_size = 50

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

print('Train size: {}'.format(len(train_loader)))


Train size: 1585


In [0]:
import torch
from torchvision import models

use_cuda = torch.cuda.is_available()

import time
tf = time.time()  

#model = LeNet()
model = models.resnet18(pretrained=True)


if use_cuda:
   model = model.cuda()
    
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)

criterion = nn.CrossEntropyLoss()
predicts=[]
trainloss = []
testloss = []

for epoch in range(10):
    # trainning
    ave_loss = 0

    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        if use_cuda:
            print(x.shape)
            x, target = x.cuda(), target.cuda()
        out = model(x)
        loss = criterion(out, target) 
        
        print(f'loss: {loss}')
        ave_loss = ave_loss * 0.9 + loss.data * 0.1
        loss.backward()
        optimizer.step()
        if (batch_idx+1) % 100 == 0 or (batch_idx+1) == len(train_loader):
            print ('==>>> epoch: {}, batch index: {}, train loss: {:.6f}'.format(
                epoch, batch_idx+1, ave_loss))    
            
    trainloss.append(loss.data)
    
     # testing    
    correct_cnt, ave_loss = 0, 0
    total_cnt = 0
    for batch_idx, (x, target) in enumerate(test_loader):
        x = x.float()
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        #x, target = Variable(x, volatile=True), Variable(target, volatile=True)
        out = model(x)        
        loss = criterion(out, target)
        _, pred_label = torch.max(out.data, 1)
        predicts.append(pred_label)
        total_cnt += x.data.size()[0]
        correct_cnt += (pred_label == target.data).sum()
        # smooth average
        ave_loss = ave_loss * 0.9 + loss.data * 0.1
        
        if(batch_idx+1) % 100 == 0 or (batch_idx+1) == len(test_loader):
            print ('==>>> epoch: {}, batch index: {}, test loss: {:.6f}, acc: {:.3f}'.format(
                epoch, batch_idx+1, ave_loss,float(correct_cnt)/total_cnt))
          
    testloss.append(loss.data)  

torch.save(model.state_dict(), model.name())

elapsed = time.time() - tf
      

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:00<00:00, 82990907.52it/s]


loss: 12.33039665222168
loss: 12.317448616027832
loss: 11.421674728393555
loss: 10.626917839050293
loss: 10.964181900024414
loss: 9.441967964172363
loss: 8.572966575622559
loss: 9.568422317504883
loss: 7.968695163726807
loss: 7.858793258666992
loss: 6.768931865692139
loss: 7.003999710083008
loss: 6.945380210876465
loss: 6.535125255584717
loss: 5.2747578620910645
loss: 5.725508213043213
loss: 5.30656623840332
loss: 4.049806594848633
loss: 3.8581650257110596
loss: 4.873569488525391
loss: 3.539717197418213
loss: 4.242822170257568
loss: 3.580151319503784
loss: 3.5226523876190186
loss: 4.0746612548828125
loss: 3.5000689029693604
loss: 3.0589439868927
loss: 3.397324323654175
loss: 3.605821132659912
loss: 3.6771020889282227
loss: 3.425966262817383
loss: 3.1780166625976562
loss: 3.0871570110321045
loss: 2.9692795276641846
loss: 3.445624589920044
loss: 3.1561596393585205
loss: 3.134550094604492
loss: 3.1377387046813965
loss: 2.997685432434082
loss: 3.292421817779541
loss: 2.9317758083343506
los